In [1]:
# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc=SparkContext()
#spark = SparkSession(sparkContext=sc)
spark = (SparkSession(sparkContext=sc)
    .builder
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.10.2,org.apache.hadoop:hadoop-client:2.10.2")
    .config("spark.jars.excludes", "com.google.guava:guava")
    .getOrCreate()
        )


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/15 10:28:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/15 10:28:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/15 10:28:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Aggregate functions
Two aggregate functions:

* `aggregate()`
* `aggregateByKey()`

### `aggregate(zeroValue, seqOp, combOp)`

* **zeroValue** is like a data container. Its structure should match with the data structure of the returned values from the seqOp function.
* **seqOp** is a function that takes two arguments: the first argument is the zeroValue and the second argument is an element from the RDD. The zeroValue gets updated with the returned value after every run.
* **combOp** is a function that takes two arguments: the first argument is the final zeroValue from one partition and the other is another final zeroValue from another partition.

The code below calculates the total sum of squares for **mpg** and **disp** in data set **mtcars**.

Step 1: get some data.

In [2]:
mtcars_df = spark.read.csv('data/SparkData/mtcars.csv', inferSchema=True, header=True).select(['mpg', 'disp'])
mtcars_df.take(5)

[Row(mpg=21.0, disp=160.0),
 Row(mpg=21.0, disp=160.0),
 Row(mpg=22.8, disp=108.0),
 Row(mpg=21.4, disp=258.0),
 Row(mpg=18.7, disp=360.0)]

In [3]:
mtcars_df.rdd.map(lambda x: (x[0], x[1])).take(5)

[(21.0, 160.0), (21.0, 160.0), (22.8, 108.0), (21.4, 258.0), (18.7, 360.0)]

Step 2: calculate averages of mgp and disp

In [4]:
mpg_mean = mtcars_df.select('mpg').rdd.map(lambda x: x[0]).mean()
disp_mean = mtcars_df.select('disp').rdd.map(lambda x: x[0]).mean()
print('mpg mean = ', mpg_mean, '; ' 
      'disp mean = ', disp_mean)

mpg mean =  20.090625000000003 ; disp mean =  230.721875


Step 3: build **zeroValue, seqOp** and **combOp**

We are calculating two TSS. We create a tuple to store two values.

In [5]:
zeroValue = (0, 0) 

The **z** below refers to `zeroValue`. Its values get updated after every run. The **x** refers to an element in an RDD partition. In this case, both **z** and **x** have two values.

In [ ]:
seqOp = lambda z, x: (z[0] + (x[0] - mpg_mean)**2, z[1] + (x[1] - disp_mean)**2)

The `combOp` function simply aggrate all `zeroValues` into one. 

In [ ]:
combOp = lambda px, py: ( px[0] + py[0], px[1] + py[1] )

Implement `aggregate()` function.

In [ ]:
mtcars_df.rdd.aggregate(zeroValue, seqOp, combOp)

## `aggregateByKey(zeroValue, seqOp, combOp)`

This function does similar things as `aggregate()`. The `aggregate()` aggregate all results to the very end, but aggregateByKey() merge results by key.

### Define initial values, seqOp and combOp

In [ ]:
x=['hello', 'world', 'good', 'hello']

In [ ]:
zero_value = 0
seqOp = (lambda x, y: x + y)
combOp = (lambda x, y: x + y)

In [ ]:
zz=sc.parallelize(x).map(lambda x: (x,1))

In [ ]:
zz.collect()

## Implement aggregateByKey()

In [ ]:
zz.aggregateByKey(zero_value,seqOp,combOp).collect()